In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
#data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'
#!wget $data

--2025-10-13 20:55:12--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.07s   

2025-10-13 20:55:13 (1.11 MB/s) - ‘course_lead_scoring.csv’ saved [80876/80876]



In [6]:
df = pd.read_csv('course_lead_scoring.csv')
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


Check if the missing values are presented in the features.

In [4]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [7]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [8]:
# Replace missing values in categorical features with 'NA'
categorical_features = df.select_dtypes(include=['object']).columns
df[categorical_features] = df[categorical_features].fillna('NA')

# Replace missing values in numerical features with 0.0
numerical_features = df.select_dtypes(include=['number']).columns
df[numerical_features] = df[numerical_features].fillna(0.0)

In [9]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

Question 1

What is the most frequent observation (mode) for the column industry?

In [12]:
df['industry'].mode()[0]

'retail'

Question 2. Biggest correlation

In [19]:
correlation = df['interaction_count'].corr(df['lead_score'])
print(f"Correlation coefficient between interaction_count and lead_score: {correlation}")
print()

correlation = df['number_of_courses_viewed'].corr(df['lead_score'])
print(f"Correlation coefficient between number_of_courses_viewed  and lead_score: {correlation}")
print()

correlation = df['number_of_courses_viewed'].corr(df['interaction_count'])
print(f"Correlation coefficient between number_of_courses_viewed  and interaction_count: {correlation}")
print()

correlation = df['annual_income'].corr(df['interaction_count'])
print(f"Correlation coefficient between annual_income   and interaction_count: {correlation}")
print()

Correlation coefficient between interaction_count and lead_score: 0.009888182496913084

Correlation coefficient between number_of_courses_viewed  and lead_score: -0.004878998354681257

Correlation coefficient between number_of_courses_viewed  and interaction_count: -0.023565222882888117

Correlation coefficient between annual_income   and interaction_count: 0.02703647240481436



Split the data

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
df_full_train, df_test= train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val= train_test_split(df_full_train, test_size=0.25, random_state=42)

In [22]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [23]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [24]:
del df_train['converted']
del df_val['converted']
del df_test['converted']

Question 3. Biggest MI

In [25]:
from sklearn.metrics import mutual_info_score

In [30]:
def mutual_info_converted_score(series):
    return mutual_info_score(series,y_train)

mi = df_train[categorical_features].apply(mutual_info_converted_score)
mi.sort_values(ascending = False)

lead_source          0.035396
employment_status    0.012938
industry             0.011575
location             0.004464
dtype: float64